# Bibliotecas 


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt #graficos
import matplotlib.cm as cm
import seaborn as sns #graficos
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (classification_report, confusion_matrix, roc_auc_score,  precision_score, recall_score, 
                             f1_score, accuracy_score, roc_curve, auc, make_scorer)
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Dicinário de Dados 


| **Variável**         | **Tipo**   | **Descrição**                                                                                                                                     |
|-----------------------|------------|---------------------------------------------------------------------------------------------------------------------------------------------------|
| RowNumber            | int64      | Número do registro (linhas), sem efeito na construção de modelos.                                                                                |
| CustomerId           | int64      | ID do cliente, sem efeito sobre o estudo.                                                                                                       |
| Surname              | object     | Sobrenome do cliente, sem impacto na análise.                                                                                                   |
| CreditScore          | int64      | Pontuação de crédito, pode indicar tendência de permanência de clientes com pontuação alta.                                                     |
| Geography            | object     | Localização do cliente, pode influenciar a decisão de evasão.                                                                                   |
| Gender               | object     | Gênero do cliente, possível influência na evasão.                                                                                               |
| Age                  | int64      | Idade do cliente, clientes mais velhos tendem a permanecer.                                                                                     |
| Tenure               | int64      | Anos que o cliente está no banco, clientes novos têm maior chance de evasão.                                                                    |
| Balance              | float64    | Saldo na conta, pessoas com saldos altos são menos propensas a sair.                                                                            |
| NumOfProducts        | int64      | Número de produtos adquiridos pelo cliente.                                                                                                    |
| HasCrCard            | int64      | Indica se o cliente tem cartão de crédito, clientes com cartão são menos propensos à evasão.                                                    |
| IsActiveMember       | int64      | Clientes ativos têm menor chance de evasão.                                                                                                    |
| EstimatedSalary      | float64    | Salário estimado, clientes com salários mais altos tendem a permanecer.                                                                         |
| Exited               | int64      | Indica se o cliente saiu ou não do banco, variável de predição (“churn”).                                                                       |
| Complain             | int64      | Indica se o cliente fez reclamação.                                                                                                             |
| Satisfaction Score   | int64      | Pontuação de satisfação com a resolução de reclamação.                                                                                          |
| Card Type            | object     | Tipo de cartão que o cliente possui.                                                                                                            |
| Points Earned        | int64      | Pontos ganhos pelo cliente.                                                                                                                     |


#  Análise Exploratória (EDA) & Data Prep


In [ ]:
# base de dados
base_original = pd.read_csv('/kaggle/input/Customer-Churn-Records.csv', sep=',')

#configs para nao quebrar linhas no print do  df
pd.set_option('display.expand_frame_repr', False) 
pd.set_option('display.max_columns', None)

#primeiras linhas 
base_original.head()

In [ ]:
#Dimensões da base de dados
print("Numero de linhas:", base_original.shape[0]) #10.000 linhas originais.
print("Numero de colunas:", base_original.shape[1])# 18 Colunas (variaveis) originais.

In [ ]:
#Verificando nome das colunas e tipos
base_original.dtypes

In [ ]:
#checando se há valores nulos 
base_original.isnull().sum()  

#como podemos ver não há valores nulos em nenhuma das variáveis

In [ ]:
# Resumo estatistico da base original
base_original.describe()


In [ ]:
# Limpando variavéis que não tem interferencia na analise, 
#meramente identificadoras: 	RowNumber, CustomerId e Surname

df = base_original[['CreditScore',
                    'Gender',
                    'Geography',
                    'Age','Tenure',
                    'Balance',
                   'NumOfProducts',
                    'HasCrCard',
                    'IsActiveMember',
                   'EstimatedSalary',
                    'Complain',
                    'Satisfaction Score',
                   'Card Type',
                    'Point Earned',
                    'Exited'
                   ]]


# Resumo estatístico das variáveis quantitativas
quanti = df[['EstimatedSalary', 'Balance', 'CreditScore', 'Age', 'Tenure', 'Point Earned']]
resumo_estati_quant = quanti.describe().style.format(lambda x: f'{x:,.1f}'.replace(',', 'X').replace('.', ',').replace('X', '.')) # Formatação com 1 casa decimal e separadores invertidos

resumo_estati_quant

In [ ]:
#%% Resumo estatistico das varaiveis Quali (tabelas de frequencias)
quali = df[['HasCrCard', 'IsActiveMember', 'Geography','Gender'
            ,'Complain','Exited','Card Type']]
quali = quali.astype('object')
print("------------------------------------------")
print(quali['HasCrCard'].value_counts())
print("------------------------------------------")
print(quali['IsActiveMember'].value_counts())
print("------------------------------------------")
print(quali['Geography'].value_counts())
print("------------------------------------------")
print(quali['Gender'].value_counts())
print("------------------------------------------")
print(quali['Complain'].value_counts())
print("------------------------------------------")
print(quali['Card Type'].value_counts())
print("------------------------------------------")
print(quali['Exited'].value_counts())
print("------------------------------------------")

In [ ]:
# Verificando tipos das variaveis quali (para morrer de certeza que estao no formato qualitativo, categorico)
quali.dtypes

In [ ]:
# Frequencia das variaveis categoricas 
#%%Analises gráficas: variaveis Categóricas 


def add_value_labels(ax):#funcao que adc rótulos de dados com fundo arredondado nas barras do gráfico
    for p in ax.patches:
        height = p.get_height()
        color = p.get_facecolor()  # Obtém a cor da barra
        # rótulo no centro da barra com fundo da mesma cor da barra e bordas arredondadas
        ax.text(p.get_x() + p.get_width() / 2., height / 2.,
                f'{int(height)}',  # Formata o valor para int
                ha='center', va='center', fontsize=20, color='white', fontweight='bold',
                bbox=dict(facecolor=color, edgecolor='none', alpha=0.7,
                          boxstyle='round,pad=0.4', linewidth=1))

plt.figure(figsize=(20, 25))


#plt.suptitle('Frequência absoluta\n das variaveis qualitativas', fontsize=45)

# Geography
plt.subplot(5, 2, 1)
ax1 = plt.gca()
ax1.set_title('Geography', fontsize=22, fontweight='bold')
sns.countplot(x='Geography', palette='viridis', data=base_original, ax=ax1)
plt.xlabel('') 
plt.ylabel('') 
plt.xticks(fontsize=15, rotation=0,fontweight='bold')
plt.yticks([])
add_value_labels(ax1)

# Gender
plt.subplot(5, 2, 2)
ax2 = plt.gca()
ax2.set_title('Gender', fontsize=22, fontweight='bold')
sns.countplot(x='Gender', palette='viridis', data=base_original, ax=ax2)
plt.xlabel('') 
plt.ylabel('') 
plt.xticks(fontsize=15, rotation=0,fontweight='bold')
plt.yticks([])
add_value_labels(ax2)
# Complain
plt.subplot(5, 2, 3)
ax10 = plt.gca()
ax10.set_title('Complain', fontsize=22, fontweight='bold')
sns.countplot(x='Complain', palette='viridis', data=base_original, ax=ax10)
plt.xlabel('') 
plt.ylabel('') 
plt.xticks(fontsize=15, rotation=0,fontweight='bold')
plt.yticks([])
add_value_labels(ax10)

# HasCrCard
plt.subplot(5, 2, 4)
ax5 = plt.gca()
ax5.set_title('HasCrCard', fontsize=22, fontweight='bold')
sns.countplot(x='HasCrCard', palette='viridis', data=base_original, ax=ax5)
plt.xlabel('') 
plt.ylabel('') 
plt.xticks(fontsize=15, rotation=0,fontweight='bold')
plt.yticks([])
add_value_labels(ax5)

# IsActiveMember
plt.subplot(5, 2, 5)
ax6 = plt.gca()
ax6.set_title('IsActiveMember', fontsize=22, fontweight='bold')
sns.countplot(x='IsActiveMember', palette='viridis', data=base_original, ax=ax6)
plt.xlabel('') 
plt.ylabel('') 
plt.xticks(fontsize=15, rotation=0,fontweight='bold')
plt.yticks([])
add_value_labels(ax6)

# Card Type
plt.subplot(5, 2, 6)
ax10 = plt.gca()
ax10.set_title('Card Type', fontsize=22, fontweight='bold')
sns.countplot(x='Card Type', palette='viridis', data=base_original, ax=ax10)
plt.xlabel('') 
plt.ylabel('') 
plt.xticks(fontsize=15, rotation=0,fontweight='bold')
plt.yticks([])
add_value_labels(ax10)

# Exited
plt.subplot(5, 2, 7)
ax7 = plt.gca()
ax7.set_title('Exited: churn variable', fontsize=22, fontweight='bold')
custom_palette = ['green', 'red']
sns.countplot(x='Exited', palette=custom_palette, data=base_original, ax=ax7)
plt.xlabel('') 
plt.ylabel('') 
plt.xticks(fontsize=15, rotation=0,fontweight='bold')
plt.yticks([])
add_value_labels(ax7)


ax7.set_xticks([0, 1])  # Define os ticks manualmente
ax7.set_xticklabels(['Não', 'Sim'], fontsize=15, fontweight='bold')  # Rótulos personalizados
add_value_labels(ax7)


# ajusta a distância entre os gráficos
plt.subplots_adjust(hspace=0.3, wspace=0.1)




In [ ]:
#%% Variável TARGET em relação as demais variáveis 

plt.figure(figsize=(20, 25))

def add_legend(ax):
    """Adiciona a legenda no canto superior direito e garante que os rótulos sejam exibidos"""
    handles, labels = ax.get_legend_handles_labels()
    if not handles:
        # Se não houver handles, adicione manualmente
        handles = [plt.Rectangle((0,0),1,1, color=c) for c in ['green', 'red']]
        labels = ['Not Exited', 'Exited']
    # Adiciona a legenda fora da área das barras
    ax.legend(handles, labels, loc='upper left', fontsize=14, title='Exited', title_fontsize='13',
              bbox_to_anchor=(1.0, 1))  # Ajusta a posição da legenda para fora das barras

# Geography
plt.subplot(5, 2, 1)
counts = base_original.groupby(['Geography', 'Exited']).size().unstack().fillna(0)
ax = counts.plot(kind='bar', stacked=True, color=['green', 'red'], ax=plt.gca())  
plt.title('Exited by Geography', fontsize=22, fontweight='bold')
plt.xlabel('Geography', fontsize=16)
plt.ylabel('', fontsize=16)
plt.xticks(fontsize=14, rotation=0,fontweight='bold')
plt.yticks(fontsize=14)
add_legend(ax)

# Gender
plt.subplot(5, 2, 2)
counts = base_original.groupby(['Gender', 'Exited']).size().unstack().fillna(0)
ax = counts.plot(kind='bar', stacked=True, color=['green', 'red'], ax=plt.gca()) 
plt.title('Exited by Gender', fontsize=22, fontweight='bold')
plt.xlabel('Gender', fontsize=16)
plt.ylabel('', fontsize=16)
plt.xticks(fontsize=14, rotation=0,fontweight='bold')
plt.yticks(fontsize=14)
add_legend(ax)

# NumOfProducts
plt.subplot(5, 2, 3)
counts = base_original.groupby(['NumOfProducts', 'Exited']).size().unstack().fillna(0)
ax = counts.plot(kind='bar', stacked=True, color=['green', 'red'], ax=plt.gca())  
plt.title('Exited by NumOfProducts', fontsize=22, fontweight='bold')
plt.xlabel('NumOfProducts', fontsize=16)
plt.ylabel('', fontsize=16)
plt.xticks(fontsize=14, rotation=0,fontweight='bold')
plt.yticks(fontsize=14)
add_legend(ax)

# HasCrCard
plt.subplot(5, 2, 4)
counts = base_original.groupby(['HasCrCard', 'Exited']).size().unstack().fillna(0)
ax = counts.plot(kind='bar', stacked=True, color=['green', 'red'], ax=plt.gca())  
plt.title('Exited by HasCrCard', fontsize=22, fontweight='bold')
plt.xlabel('HasCrCard', fontsize=16)
plt.ylabel('', fontsize=16)
plt.xticks(fontsize=14, rotation=0,fontweight='bold')
plt.yticks(fontsize=14)
add_legend(ax)

# IsActiveMember
plt.subplot(5, 2, 5)
counts = base_original.groupby(['IsActiveMember', 'Exited']).size().unstack().fillna(0)
ax = counts.plot(kind='bar', stacked=True, color=['green', 'red'], ax=plt.gca()) 
plt.title('Exited by IsActiveMember', fontsize=22, fontweight='bold')
plt.xlabel('IsActiveMember', fontsize=16)
plt.ylabel('', fontsize=16)
plt.xticks(fontsize=14, rotation=0,fontweight='bold')
plt.yticks(fontsize=14)
add_legend(ax)

# Complain
plt.subplot(5, 2, 6)
counts = base_original.groupby(['Complain', 'Exited']).size().unstack().fillna(0)
ax = counts.plot(kind='bar', stacked=True, color=['green', 'red'], ax=plt.gca())  
plt.title('Exited by Complain', fontsize=22, fontweight='bold')
plt.xlabel('Complain', fontsize=16)
plt.ylabel('', fontsize=16)
plt.xticks(fontsize=14, rotation=0,fontweight='bold')
plt.yticks(fontsize=14)
add_legend(ax)

# Satisfaction Score
plt.subplot(5, 2, 7)
counts = base_original.groupby(['Satisfaction Score', 'Exited']).size().unstack().fillna(0)
ax = counts.plot(kind='bar', stacked=True, color=['green', 'red'], ax=plt.gca())  
plt.title('Exited by Satisfaction Score', fontsize=22, fontweight='bold')
plt.xlabel('Satisfaction Score', fontsize=16)
plt.ylabel('', fontsize=16)
plt.xticks(fontsize=14, rotation=0,fontweight='bold')
plt.yticks(fontsize=14)
add_legend(ax)

# Card Type
plt.subplot(5, 2, 8)
counts = base_original.groupby(['Card Type', 'Exited']).size().unstack().fillna(0)
ax = counts.plot(kind='bar', stacked=True, color=['green', 'red'], ax=plt.gca())  
plt.title('Exited by Card Type', fontsize=22, fontweight='bold')
plt.xlabel('Card Type', fontsize=16)
plt.ylabel('', fontsize=16)
plt.xticks(fontsize=14, rotation=0,fontweight='bold')
plt.yticks(fontsize=14)
add_legend(ax)

# Ajustar a distância entre os gráficos
plt.subplots_adjust(hspace=0.7, wspace=0.3)

# Variaveis Dummies e Correlações 

In [ ]:
# Dumizando

# Transformando colunas específicas em tipo object usando .loc para evitar o Warning
cols_to_transform = ['HasCrCard', 'IsActiveMember', 'Geography', 'Gender', 'Card Type']
df.loc[:, cols_to_transform] = df[cols_to_transform].astype('object')



# Gerando as dummies
df_dummies = pd.get_dummies(df,
                           columns=['HasCrCard',
                                    'IsActiveMember', 
                                    'Geography',
                                    'Gender',
                                    'Card Type'],
                           dtype=int,
                           drop_first=False)


#Transformando Target em numérica
df_dummies['Exited'] = df_dummies['Exited'].astype('int64')



#verificando tipos gerados
df_dummies.dtypes

In [ ]:
#%%Analises gráficas: correlação das variaveis Numéricas 

correlation_matrix = df_dummies.corr().round(2)
correlation_matrix

# Mapa de calor das variaveis quanti
plt.figure(figsize=(30, 20))
heatmap = sns.heatmap(correlation_matrix, annot=True, fmt=".2f",
                      cmap=plt.cm.Blues,
                      annot_kws={'size': 15}, vmin=-1, vmax=1)
heatmap.set_xticklabels(heatmap.get_xticklabels(), fontsize=17)
heatmap.set_yticklabels(heatmap.get_yticklabels(), fontsize=17)
cbar = heatmap.collections[0].colorbar
cbar.ax.tick_params(labelsize=17)
plt.title('Correlação das Variáveis Quantitativas', fontsize=25)
plt.show()


#apresentaram correlacao alta, para evitar multicolinearidade foram removidas 
df_dummies = df_dummies.drop(columns=['HasCrCard_0','IsActiveMember_0','Gender_Female','Complain'])


# Separação Treino e Teste

In [ ]:
#X ---> Variáveis explicativas 

#Y ---> Evento de estudo (variável TARGET)

X = df_dummies .drop('Exited', axis=1)

y =  df_dummies['Exited']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



# Verificando a proporção de eventos de churn (TARGET) nas bases de TREINO e TESTE 

# Contando os valores 
churn_counts_train = y_train.value_counts()
churn_counts_test = y_test.value_counts()

# Criando o plot com subplots lado a lado
fig, axs = plt.subplots(1, 2, figsize=(20, 10))

# Adicionando título geral ao plot
fig.suptitle('Proporção da Variável Churn entre Treino e Teste', fontsize=35)

#definindo a paleta de cor padrao a ser usada nos dois graficos 
cmap = plt.get_cmap('viridis', 2)

# Gráfico da base de treino
bars_train = axs[0].bar(churn_counts_train.index, churn_counts_train.values, color=cmap(range(2)))
axs[0].set_title('Base de Treino', fontsize=25)
axs[0].set_xlabel('Churn', fontsize=20)
axs[0].set_ylabel('Contagem', fontsize=20)
axs[0].set_xticks([0, 1])
axs[0].set_xticklabels(['0', '1'], fontsize=20)

# Ocultando os valores do eixo y
axs[0].set_yticklabels([])

# Adicionando rótulos de dados nas barras da base de treino com valor absoluto e percentual
total_train = churn_counts_train.sum()
for bar in bars_train:
    count = int(bar.get_height())
    percentage = round(count / total_train * 100)  # Arredonda a porcentagem
    label = f'{count} ({percentage}%)'  # Exibe o valor absoluto e o percentual
    axs[0].text(bar.get_x() + bar.get_width() / 2, bar.get_height() / 2, 
                label, ha='center', color='gray', fontsize=25, weight='bold')

# Gráfico da base de teste
bars_test = axs[1].bar(churn_counts_test.index, churn_counts_test.values, color=cmap(range(2)))
axs[1].set_title('Base de Teste', fontsize=25)
axs[1].set_xlabel('Churn', fontsize=20)
axs[1].set_ylabel('Contagem', fontsize=20)
axs[1].set_xticks([0, 1])
axs[1].set_xticklabels(['0', '1'], fontsize=20)

# Ocultando os valores do eixo y
axs[1].set_yticklabels([])

# Adicionando rótulos de dados nas barras da base de teste com valor absoluto e percentual
total_test = churn_counts_test.sum()
for bar in bars_test:
    count = int(bar.get_height())
    percentage = round(count / total_test * 100)  # Arredonda a porcentagem
    label = f'{count} ({percentage}%)'  # Exibe o valor absoluto e o percentual
    axs[1].text(bar.get_x() + bar.get_width() / 2, bar.get_height() / 2, 
                label, ha='center', color='gray', fontsize=25, weight='bold')

# Ajustar o layout para evitar sobreposição
plt.tight_layout(rect=[0, 0, 1, 0.95])  # Deixa espaço para o título principal
plt.show()

In [ ]:
#%% Testando Multicolinearidade na base de treino

#Aqui é só para morrer de certeza, para evitar que o modelo treine errado ou sofra com multicolinearidade. 

# todo esse processo de tirar as variaveis que tem alta correlacao com a variavel target pode ser analisado se realmente é necessario, a depender do modelo 
# existem modelos que capturam bem isso e não são afetados pela multicolinearidade, nesse estudo vamos testar das duas formas, com e sem essas variáveis. 

teste_multco_treino = pd.concat([X_train,y_train], axis = 1)

correlation_matrix_treino = teste_multco_treino.corr().round(2)
correlation_matrix_treino

# Mapa de calor das variaveis quanti (SEM COMPLAIN)
plt.figure(figsize=(30, 20))
heatmap = sns.heatmap(correlation_matrix_treino, annot=True, fmt=".2f",
                      cmap=plt.cm.viridis_r, # é importante notar que a paleta de cores viridis (ou viridis_r para o inverso de cores) é uma paleta especial 
                                             # para facilitar a visualizacao por pessoas com dificuldades, como os daltonicos. 
                      annot_kws={'size': 15}, vmin=-1, vmax=1)
heatmap.set_xticklabels(heatmap.get_xticklabels(), fontsize=17)
heatmap.set_yticklabels(heatmap.get_yticklabels(), fontsize=17)
plt.title('Correlação das Variáveis Quantitativas na Base de Treino',fontsize=25)
cbar = heatmap.collections[0].colorbar
cbar.ax.tick_params(labelsize=17)
plt.show()

# Analise e tratamento de Outliers 

In [ ]:
#%% analise de outliers das variaveis na base de treino 


#avariaveis analisadas (numéricas)
variaveis = [
    'CreditScore',
    'Age',
    'Tenure',
    'Balance',
    'NumOfProducts',
    'EstimatedSalary',
    'Satisfaction Score',
    'Point Earned'
]

# subplots
plt.figure(figsize=(12, 8))

# boxplots separados para cada variável
for i, var in enumerate(variaveis):
    plt.subplot(3, 3, i + 1)  # 3 linhas e 3 colunas
    sns.boxplot(y=teste_multco_treino[var],
               boxprops=dict(facecolor='lightblue'))  # Cor interna do boxplot)  
    plt.title(f'Boxplot {var}', fontsize=12)


#  título geral
plt.suptitle('Análise de Outliers nas Variáveis(treino) - antes de "winsorization" ', fontsize=20)

# Ajuste de layout
plt.tight_layout(rect=[0, 0, 1, 0.95])  # Ajuste de layout sem sobrepor o título
plt.show()




#aplicando procedimento de truncamento ou winsorization nos outliers

# ---> substitui os outliers pelos valores dos limites inferior e superior, 
#de acordo com a posicao de cada outliers na distribuição dos dados

# Função para tratar outliers
def tratar_outliers(df, coluna):
    Q1 = df[coluna].quantile(0.25)
    Q3 = df[coluna].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    
    # Substitui outliers pelo limite inferior ou superior
    df[coluna] = np.where(df[coluna] < limite_inferior, limite_inferior, df[coluna])
    df[coluna] = np.where(df[coluna] > limite_superior, limite_superior, df[coluna])

# Aplicando a função nas variáveis 
variaveis_para_tratar = ['Age', 'CreditScore', 'NumOfProducts']

for variavel in variaveis_para_tratar:
    tratar_outliers(teste_multco_treino, variavel)
    
# subplot
plt.figure(figsize=(12, 8))

# boxplots separados para cada variável
for i, var in enumerate(variaveis):
    plt.subplot(3, 3, i + 1)  # 3 linhas e 3 colunas
    sns.boxplot(y=teste_multco_treino[var],
               boxprops=dict(facecolor='green'))  # Cor interna do boxplot)  # Usar o nome da variável diretamente
            
    plt.title(f'Boxplot {var}', fontsize=12)

# título geral
plt.suptitle('Análise de Outliers nas Variáveis(treino) - depois de "winsorization" ', fontsize=20)

# Ajustando layout
plt.tight_layout(rect=[0, 0, 1, 0.95])  # Ajusta o layout sem sobrepor o título
plt.show()

In [ ]:
#%% Separando novamente as bases de treino e teste depois de tratar os outliers APENAS na base de TREINO  

#treino

X_train=teste_multco_treino.drop('Exited', axis=1)
y_train=teste_multco_treino['Exited']

#teste
base_corrige_teste = pd.concat([X_test,y_test], axis = 1)
X_test=base_corrige_teste.drop('Exited', axis=1)
y_test=base_corrige_teste['Exited']


print('---------------------------------------')
print(X_train.isnull().sum())
print('---------------------------------------')
print('---------------------------------------')
print(X_train.dtypes)
print('---------------------------------------')
print('---------------------------------------')
print(y_train.value_counts())
print('---------------------------------------')

# Modelagem: aplicando MLP 

Ver se vale a pena testar modelo com as variaveis quev foram removidas por multicolinearidade. 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from datetime import datetime, timedelta

print('------------------------')
print("MLP - Multi-Layer Perceptron ")
print('------------------------')
# início
start_time_utc = datetime.utcnow() - timedelta(hours=3)
print('------------------------')
print("Início:", start_time_utc)
print('------------------------')

# Exemplo de dados fictícios (use seus dados aqui)
# X_train, X_test, y_train, y_test são os dados de treino e teste já definidos.
# Para fins de exemplo, vamos assumir que os dados estão prontos.

# Normalizando os dados de treinamento
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Aplicando SMOTE para oversampling
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_scaled, y_train)

# Convertendo para tensores PyTorch
X_train_tensor = torch.tensor(X_train_res, dtype=torch.float32).cuda()
y_train_tensor = torch.tensor(y_train_res, dtype=torch.float32).cuda()
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).cuda()
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).cuda()

# Função para criar o modelo MLP
def create_model(hidden_layer_sizes, activation='relu', solver='adam', alpha=0.00001, 
                 learning_rate='constant', learning_rate_init=0.0001, max_iter=200, batch_size=32, 
                 momentum=0.9, early_stopping=True, validation_fraction=0.1):
    class MLP(nn.Module):
        def __init__(self):
            super(MLP, self).__init__()
            self.layers = nn.ModuleList()
            input_size = X_train_tensor.shape[1]
            for units in hidden_layer_sizes:
                self.layers.append(nn.Linear(input_size, units))
                input_size = units
            self.output = nn.Linear(input_size, 1)
            self.activation_fn = torch.relu if activation == 'relu' else torch.tanh
            self.sigmoid = nn.Sigmoid()

        def forward(self, x):
            for layer in self.layers:
                x = self.activation_fn(layer(x))
            x = self.sigmoid(self.output(x))
            return x

    model = MLP().cuda()
    criterion = nn.BCELoss()

    # Ajustando o otimizador de acordo com o parâmetro solver
    if solver == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=learning_rate_init)
    elif solver == 'sgd':
        optimizer = optim.SGD(model.parameters(), lr=learning_rate_init, momentum=momentum)

    return model, criterion, optimizer

# Função de objetivo para o Optuna
def objective(trial):
    # Definição dos hiperparâmetros
    hidden_layer_sizes = trial.suggest_categorical('hidden_layer_sizes', [(50,), (100,), (50, 50)])
    activation = trial.suggest_categorical('activation', ['relu', 'tanh'])
    solver = trial.suggest_categorical('solver', ['adam', 'sgd'])
    alpha = trial.suggest_loguniform('alpha', 0.00001, 0.01)
    learning_rate = trial.suggest_categorical('learning_rate', ['constant', 'adaptive'])
    learning_rate_init = trial.suggest_categorical('learning_rate_init', [0.0001, 0.01])
    max_iter = trial.suggest_categorical('max_iter', [200, 500])
    batch_size = trial.suggest_categorical('batch_size', [32, 64])
    momentum = trial.suggest_categorical('momentum', [0.9, 0.95])
    early_stopping = trial.suggest_categorical('early_stopping', [True])
    validation_fraction = trial.suggest_categorical('validation_fraction', [0.1])

    # Criando o modelo
    model, criterion, optimizer = create_model(
        hidden_layer_sizes=hidden_layer_sizes, activation=activation, solver=solver, 
        alpha=alpha, learning_rate=learning_rate, learning_rate_init=learning_rate_init,
        max_iter=max_iter, batch_size=batch_size, momentum=momentum, early_stopping=early_stopping, 
        validation_fraction=validation_fraction
    )

    # Treinamento
    for epoch in range(max_iter):
        model.train()
        optimizer.zero_grad()
        output = model(X_train_tensor).squeeze()
        loss = criterion(output, y_train_tensor)
        loss.backward()
        optimizer.step()

    # Avaliação
    model.eval()
    y_pred_prob_test = model(X_test_tensor).cpu().detach().numpy().squeeze()
    y_pred_test = (y_pred_prob_test > 0.5).astype(int)

    # Cálculo das métricas
    accuracy = accuracy_score(y_test, y_pred_test)
    precision = precision_score(y_test, y_pred_test)
    recall = recall_score(y_test, y_pred_test)
    f1 = f1_score(y_test, y_pred_test)
    roc_auc = roc_auc_score(y_test, y_pred_prob_test)

    # Exibir as métricas
    print('------------------------')
    print("\nMétricas do Conjunto de Teste para o Modelo Neural (Cenário 3):")
    print(f"ROC AUC: {roc_auc:.4f}")
    print(f"Acurácia: {accuracy:.4f}")
    print(f"Precisão: {precision:.4f}")
    print(f"Revocação: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print('------------------------')
    print('------------------------')

    # Matriz de Confusão
    cm = confusion_matrix(y_test, y_pred_test)

    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Classe 0', 'Classe 1'], yticklabels=['Classe 0', 'Classe 1'])
    plt.title('Matriz de Confusão')
    plt.xlabel('Predito')
    plt.ylabel('Real')
    plt.show()

    # Curva ROC-AUC
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob_test)
    plt.figure(figsize=(10, 7))
    plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (F1-Score = {f1:.2f})')
    plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
    plt.title('Curva ROC-AUC')
    plt.xlabel('Taxa de Falsos Positivos')
    plt.ylabel('Taxa de Verdadeiros Positivos')
    plt.legend(loc='lower right')
    plt.show()

    return f1

# Executando a otimização
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

print(f"Melhores parâmetros: {study.best_params}")

# início
print('------------------------')
print("Início:", start_time_utc)
print('------------------------')

# fim
end_time_utc = datetime.utcnow() - timedelta(hours=3)
print('------------------------')
print("Fim:", end_time_utc)
print('------------------------')
